In [25]:
import notebook_setup
from notebook_setup import MODELS_DIR

import numpy as np
import torch 
from pathlib import Path
import plotly.graph_objects as go

from gpmdm import GPMDM, GPMDM_PF
from dataset_utils.mocap_labels import WALK_TRIALS_TRAIN, WALK_TRIALS_TEST, RUN_TRIALS_TRAIN, RUN_TRIALS_TEST
import dataset_utils.select_joints as select_joints

In [26]:
model_path = MODELS_DIR / 'gpmdm' / 'gpmdm_2024_12_04.pth'
gpdm = GPMDM.load(model_path)


Model and hyperparameters correctly loaded


In [27]:
reduced_columns = WALK_TRIALS_TEST[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")


Number of DOFs: 35


In [28]:
TEST_TRIALS = RUN_TRIALS_TEST + WALK_TRIALS_TEST
TRUTH_LABELS = [1] * len(RUN_TRIALS_TEST) + [0] * len(WALK_TRIALS_TEST)

reduced_columns = TEST_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")

Number of DOFs: 35


In [29]:
# Markov Transition Matrix between classes
markov_transition_matrix = torch.tensor(
    [[0.9, 0.1], 
     [0.1, 0.9]])

# Build a gpdm_pf model
gpdm_pf = GPMDM_PF(gpdm, 
                   markov_switching_model=markov_transition_matrix,
                   num_particles=100,
                   n_fresh_particles_at_each_timestep=0)

In [30]:
trial_number = 36
test_trial = TEST_TRIALS[trial_number]
test_trial_label = TRUTH_LABELS[trial_number]
data_arr = test_trial.as_dataframe()[reduced_columns].to_numpy(dtype=np.float32)[::5, :]

In [31]:
gpdm_pf.reset()
correct = 0
total = 0

predicted_latent_states = torch.zeros((len(data_arr)+1, gpdm_pf.latent_dim))
predicted_latent_states[0] = gpdm_pf.current_state_mean()
total += 1

# Run the filter bank on the trial
for data in data_arr:
    gpdm_pf.update(data)
    predicted_label = gpdm_pf.get_most_likely_class()
    latest_probs = gpdm_pf.class_probabilities()
    p_walk = latest_probs[0]
    p_run = latest_probs[1]
    predicted_latent_states[total] = gpdm_pf.current_state_mean()

    if predicted_label == test_trial_label:
        correct += 1
    
    total += 1

accuracy = correct / total

print(f"True label: {test_trial_label}, Predicted label: {predicted_label}")
print(f"Accuracy: {accuracy}")

True label: 0, Predicted label: 0
Accuracy: 0.8584905660377359


In [32]:

# Assuming gpdm.get_X_for_class is your method to get the data
X0 = gpdm.get_X_for_class(0).detach().numpy()
X1 = gpdm.get_X_for_class(1).detach().numpy()
predicted_latent_states = predicted_latent_states.detach().numpy()

# Create the plot
fig = go.Figure()

# Add Class 0 data (in blue)
fig.add_trace(go.Scatter3d(
    x=X0[:, 0],
    y=X0[:, 1],
    z=X0[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue'),
    name='Class 0'
))

# Add Class 1 data (in red)
fig.add_trace(go.Scatter3d(
    x=X1[:, 0],
    y=X1[:, 1],
    z=X1[:, 2],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Class 1'
))

# Add predicted latent states (in green)
# make then a gradient of green to show the trajectory
fig.add_trace(go.Scatter3d(
    x=predicted_latent_states[:, 0],
    y=predicted_latent_states[:, 1],
    z=predicted_latent_states[:, 2],
    mode='lines+markers',
    marker=dict(size=5, color=np.linspace(0, 1, len(predicted_latent_states)), colorscale='Greens'),
    line=dict(color='green', width=2),
    name='Predicted Latent States'
))

# Update layout with axis labels and title
fig.update_layout(
    title="3D Latent Space Plot",
    scene=dict(
        xaxis_title="X-axis",
        yaxis_title="Y-axis",
        zaxis_title="Z-axis"
    )
)

# Show the interactive plot
fig.show()